Connect client


In [12]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
import numpy as np


from fastembed.embedding import DefaultEmbedding
import utils
# import numpy as np
# CONNECT TO CLIENT (FIRST RUN CONTAINER AT PORT 6333)
qdrantClient = QdrantClient(host='localhost', port=6333)

In [2]:
base_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_base.fvecs")
query_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_query.fvecs")
knn_groundtruth = utils.read_ivecs("../../dataset/siftsmall/siftsmall_groundtruth.ivecs")

Loading file: siftsmall_base.fvecs
    The dimension of the vectors in the file is: 128
    The final shape of the loaded dataset siftsmall_base.fvecs is (10000, 128).
Loading file: siftsmall_query.fvecs
    The dimension of the vectors in the file is: 128
    The final shape of the loaded dataset siftsmall_query.fvecs is (100, 128).
 Loading file: siftsmall_groundtruth.ivecs
    The dimension of the vectors in the file is: 100
    The final shape of the loaded dataset is (100, 100).


In [3]:
vector_size = 128
collection_name = "test_collection"

qdrantClient.delete_collection(collection_name=collection_name)

qdrantClient.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.EUCLID),
)


True

Fill database with base vectors (10K now)

In [4]:
batch_points = [PointStruct(id=i, vector=elem) for i, elem in enumerate(base_vectors)]


# Upload the batch
operation_info = qdrantClient.upsert(
    collection_name=collection_name,
    wait=True,
    points=batch_points
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


now look for the knn (k=100 bc its what the ground truth contains)

In [5]:
for elem in query_vectors:
    print(elem)
    break

[  1.   3.  11. 110.  62.  22.   4.   0.  43.  21.  22.  18.   6.  28.
  64.   9.  11.   1.   0.   0.   1.  40. 101.  21.  20.   2.   4.   2.
   2.   9.  18.  35.   1.   1.   7.  25. 108. 116.  63.   2.   0.   0.
  11.  74.  40. 101. 116.   3.  33.   1.   1.  11.  14.  18. 116. 116.
  68.  12.   5.   4.   2.   2.   9. 102.  17.   3.  10.  18.   8.  15.
  67.  63.  15.   0.  14. 116.  80.   0.   2.  22.  96.  37.  28.  88.
  43.   1.   4.  18. 116.  51.   5.  11.  32.  14.   8.  23.  44.  17.
  12.   9.   0.   0.  19.  37.  85.  18.  16. 104.  22.   6.   2.  26.
  12.  58.  67.  82.  25.  12.   2.   2.  25.  18.   8.   2.  19.  42.
  48.  11.]


In [6]:
knn_groundtruth[0]

array([2176, 3752,  882, 4009, 2837,  190, 3615,  816, 1045, 1884,  224,
       3013,  292, 1272, 5307, 4938, 1295,  492, 9211, 3625, 1254, 1292,
       1625, 3553, 1156,  146,  107, 5231, 1995, 9541, 3543, 9758, 9806,
       1064, 9701, 4064, 2456, 2763, 3237, 1317, 3530,  641, 1710, 8887,
       4263, 1756,  598,  370, 2776,  121, 4058, 7245, 1895,  124, 8731,
        696, 4320, 4527, 4050, 2648, 1682, 2154, 1689, 2436, 2005, 3210,
       4002, 2774,  924, 6630, 3449, 9814, 3515, 5375,  287, 1038, 4096,
       4094,  942, 4321,  123, 3814,   97, 4293,  420, 9734, 1916, 2791,
        149, 6139, 9576, 6837, 2952, 3138, 2890, 3066, 2852,  348, 3043,
       3687], dtype=int32)

In [7]:
# for vec in query_vectors:
#     search_result = qdrantClient.search(
#         collection_name="test_collection", query_vector=[0.0 for _ in range(vector_size)], limit=100
#     )
#     break

for vec in query_vectors:
    search_result = qdrantClient.search(
        collection_name="test_collection", query_vector=vec, limit=100
    )
    break




In [15]:
import numpy as np

res = [elem.id for elem in search_result]
res = np.array(res, dtype=np.int32)

intersection = np.intersect1d(knn_groundtruth[0], res)
intersection

array([  97,  107,  121,  123,  124,  146,  149,  190,  224,  287,  292,
        348,  370,  420,  492,  598,  641,  696,  816,  882,  924,  942,
       1038, 1045, 1064, 1156, 1254, 1272, 1292, 1295, 1317, 1625, 1682,
       1689, 1710, 1756, 1884, 1895, 1916, 1995, 2005, 2154, 2176, 2436,
       2456, 2648, 2763, 2774, 2776, 2791, 2837, 2852, 2890, 2952, 3013,
       3043, 3066, 3138, 3210, 3237, 3449, 3515, 3530, 3543, 3553, 3615,
       3625, 3687, 3752, 3814, 4002, 4009, 4050, 4058, 4064, 4094, 4096,
       4263, 4293, 4320, 4321, 4527, 4938, 5231, 5307, 5375, 6139, 6630,
       6837, 7245, 8731, 8887, 9211, 9541, 9576, 9701, 9734, 9758, 9806,
       9814], dtype=int32)

In [16]:
knn_indices = utils.top_k_neighbors(query_vectors, base_vectors, k=100)

In [17]:
intersection = np.intersect1d(knn_indices[0], res)
len(intersection)

100